# Agent development colab


This notebook presents a prompt engineering view of the Concordia agent development process, making it easier to configure within the Concordia environment. For coding proficient users, a higher level of customization and development will be possible by developing agents directly in the agent factory.


To begin, make a copy of this colab and save to google drive. This will impact where the agent file is created that you will eventually submit.


Agent Factory: https://github.com/google-deepmind/concordia/tree/main/concordia/factory/agent

Additional Agent Tutorials: https://github.com/google-deepmind/concordia/tree/main/examples/tutorials


<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/tutorials/agent_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup and imports

In [1]:
import sys
import os

project_root = "C:/Users/TomFi/Desktop/Cours/Concordia"
# Add root to python path.
if project_root not in sys.path:
  sys.path.append(project_root)

In [2]:
import datetime
import importlib
import numpy as np

from IPython import display

from concordia.language_model import call_limit_wrapper
from concordia.language_model import utils
from concordia.utils import measurements as measurements_lib

## Language Model setup

In [3]:
from google.auth import default
from google.auth.exceptions import DefaultCredentialsError
from google.cloud import aiplatform
import sentence_transformers

try:
    credentials, project = default()
except DefaultCredentialsError as e:
    print(f"Error: {e}")
    print("Please set up Application Default Credentials as described in the documentation.")
    raise

from concordia.language_model import google_cloud_custom_model

_embedder_model = sentence_transformers.SentenceTransformer(
    'sentence-transformers/all-mpnet-base-v2')
embedder = lambda x: _embedder_model.encode(x, show_progress_bar=False)

endpoint_id = '7512339529500459008' #@param {type: 'string'}
project_id = '1085311675177' #@param {type: 'string'}
region = 'us-central1' #@param {type: 'string'}

if not endpoint_id:
  raise ValueError('model endpoint id is required')
if not project_id:
  raise ValueError('A project id is required.')
if not region:
  raise ValueError('Region information is required.')

model = google_cloud_custom_model.VertexAI(endpoint_id=endpoint_id,
      project_id=project_id,
      location=region,)

c:\Users\TomFi\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
mems = "Recent observations of Henry Green:[13 May 2015 10:00:00] [observation] Here is the conversation from the beginning:[13 May 2015 10:00:00] [observation] European football cup is on.[13 May 2015 10:00:00] [observation] Games are best watched in pubs with a lot of friends.[13 May 2015 10:00:00] Games are best watched in pubs with a lot of friends.[13 May 2015 10:00:00] [observation] The year is 2015. This week is the European football cup.[14 May 2015 10:00:00] [observation] The aroma of freshly brewed coffee and artisan pastries fills the air. Henry Green, Harry Davies sit at a long wooden table under a striped awning, their laughter mingling with the chatter of the bustling market. A street musician strums a guitar nearby, adding a bohemian touch to the scene. Henry Green just arrived.[14 May 2015 10:00:00] [observation] Tonight is the night of the game between Russia and England. Friends are going to watch the game at a pub, but they are not sure which pub to go to.[14 May 2015 10:00:00] [observation] Harry Davies  --Right then lads, The Golden Fleece it is. Know theyll have all the matches on [14 May 2015 10:00:00] [observation] Henry Green  -- Hold on a second, the Duke of York is far better,  they usually have the best atmosphere and... [14 May 2015 18:00:00] [observation] [14 May 2015 18:00:00] [observation] Henry Green went to The Duke of York. Harry Davies went to The Golden Fleece.  Henry Green watched the game at their favourite pub. None of Henry Greens friends showed up, it couldnt have been worse! Overall, Henry Green had the worst time ever.[14 May 2015 18:00:00] [observation] Henry Green realises it is time to go watch the game at a pub. [15 May 2015 10:00:00] [observation] Sunlight dances on the water as Henry Green, Harry Davies cycle along the towpath of Regents Canal. They pause to admire the colorful houseboats and wave at fellow cyclists. The gentle sound of water lapping against the canal banks creates a peaceful atmosphere. Henry Green just arrived.[15 May 2015 10:00:00] [observation] Harry Davies  -- Phone Henry.  Make sure he knows the Golden Fleece is still the plan. [15 May 2015 10:00:00] [observation] Henry Green  -- I probably shouldnt tell Harry that now, but Id much prefer The Duke of York. [15 May 2015 10:00:00] [observation] Tonight is the night of the game between Slovenia and Czech Republic. Friends are going to watch the game at a pub, but they are not sure which pub to go to.[15 May 2015 18:00:00] [observation] Henry Green realises it is time to go watch the game at a pub. [15 May 2015 18:00:00] [observation] [15 May 2015 18:00:00] [observation] Henry Green went to The Duke of York. Harry Davies went to The Golden Fleece.  Henry Green watched the game at their favourite pub. None of Henry Green's friends showed up, it couldn't have been worse! Overall, Henry Green had the worst time ever.[16 May 2015 10:00:00] [observation] The sun peeks through the morning mist as Henry Green, Harry Davies gather near the entrance of Hackney Marshes. They stretch and laugh, their colorful running attire contrasting with the green expanse. A few dog walkers pass by, their furry companions excitedly sniffing the air. Henry Green just arrived.[16 May 2015 10:00:00] [observation] Tonight is the night of the game between Iceland and Cyprus. Friends are going to watch the game at a pub, but they are not sure which pub to go to.[16 May 2015 10:00:00] [observation] Henry Green  -- Harry, fancy a pint at The Duke of York to watch the match? Theyll have a good crowd in there. [16 May 2015 10:00:00] [observation] Harry Davies   -- The Golden Fleece it is then!  Plenty of room for everyone there. [16 May 2015 18:00:00] [observation] Henry Green realises it is time to go watch the game at a pub. [16 May 2015 18:00:00] [observation]Current time: 2015-05-16 18:00:00.Overarching goal: Have a good time. To have a good time, Henry Green would like to watch the game in the same pub as Henry Green, Harry Davies. Henry Green would prefer everyone went to The Duke of York. Observation: [16 May 2015 18:00:00] [observation] Henry Green realises it is time to go watch the game at a pub. [16 May 2015 18:00:00] [observation] Summary of recent observations: [15 May 2015 18:00:00  - 16 May 2015  18:00:00]:  Henry Green Recalled memories and observations: [14 May 2015 10:00:00] [observation] Henry Green  -- Hold on a second, the Duke of York is far better,  they usually have the best atmosphere and...[15 May 2015 10:00:00] [observation] Henry Green  -- I probably shouldnt tell Harry that now, but Id much prefer The Duke of York. [16 May 2015 10:00:00] [observation] Henry Green  -- Harry, fancy a pint at The Duke of York to watch the match? Theyll have a good crowd in there."
agent_name = 'Henry Green'
context = ''
question = 'Summarize the observations above into one or two sentences.'
from concordia.document import interactive_document

prompt = interactive_document.InteractiveDocument(model)
prompt.statement(context + '\n')
prompt.statement(
    f'Recent observations of {agent_name}:\n' + f'{mems}\n'
)
result = (
    agent_name
    + ' '
    + prompt.open_question(
        question,
        answer_prefix=f'{agent_name} ',
        max_tokens=8191,
    )
)
# print(f'PROMPT : {prompt._model_view.text()}')
# print(result)
# model.sample_text(prompt = 'who is the president of France ?')
# model.sample_text(prompt=prompt)


--------- ASKING A QUESTION --------- 
Context given to LLM : 


Recent observations of Henry Green:
Recent observations of Henry Green:[13 May 2015 10:00:00] [observation] Here is the conversation from the beginning:[13 May 2015 10:00:00] [observation] European football cup is on.[13 May 2015 10:00:00] [observation] Games are best watched in pubs with a lot of friends.[13 May 2015 10:00:00] Games are best watched in pubs with a lot of friends.[13 May 2015 10:00:00] [observation] The year is 2015. This week is the European football cup.[14 May 2015 10:00:00] [observation] The aroma of freshly brewed coffee and artisan pastries fills the air. Henry Green, Harry Davies sit at a long wooden table under a striped awning, their laughter mingling with the chatter of the bustling market. A street musician strums a guitar nearby, adding a bohemian touch to the scene. Henry Green just arrived.[14 May 2015 10:00:00] [observation] Tonight is the night of the game between Russia and England. Frie

# Test LLM prompting with context

In [5]:
memory_text = ["Observation: [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane just arrived at the saloon after a hard day of work.","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane's possessions: {'coin': 5.0}","[02 Oct 1868 20:00:00] [observation]   Rutherford Coltrane is at the saloon, having just arrived after a hard day of work.","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane spent -0.75 coin on daily expenses.","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane ordered a glass of sarsaparilla and, while seated at the bar, inquired, '[direct quote] What’s the latest news from the railroad?' He engaged the bartender in conversation about his own experiences from the day, discussing the challenges he faced at the outpost, which led the bartender to share gossip about rising tensions among workers and Abner Stokes's efforts to rally support for a strike, making Rutherford aware of the brewing unrest and its potential implications for his safety and the railroad's stability.  As a result Rutherford Coltrane, Abner Stokes are in conversation. Here is the conversation from the beginning:","[02 Oct 1868 20:00:00] [observation] Rutherford Coltrane -- 'I understand the frustrations you all are feeling. It's important we find a way to discuss these issues without resorting to conflict. How can we work together to ensure that everyone's voice is heard?'","[02 Oct 1868 20:00:00] [observation] Abner Stokes -- 'We can't just sit back and hope for the best, Rutherford! The time for talk is over. It's time for action, and if you’re not willing to stand with us, then you’re part of the problem. We need to make our voices impossible to ignore!'","[02 Oct 1868 20:00:10] [observation] Abner Stokes -- 'Rutherford, your well-meaning words won't change a thing if we don't take a stand! We can't afford to tiptoe around the management's feelings while our livelihoods are at stake. We need to be loud, we need to be fierce, and we need to make it clear that we won't be silenced any longer!","[02 Oct 1868 20:00:10] [observation] Rutherford Coltrane -- 'I hear your frustrations, Abner, and I respect your passion. But let’s consider how we can present our demands in a way that compels management to listen. We need to be united and strategic if we want to bring about real change.","[02 Oct 1868 20:00:20] [observation] Rutherford Coltrane -- 'Abner, I hear your urgency, but let us not forget the power of unity in our approach. Perhaps we can gather the workers for a discussion, to ensure that our demands are clear and compelling. Together, we can make our voices resonate without resorting to conflict.","[02 Oct 1868 20:00:20] [observation] Abner Stokes -- 'Rutherford, we can't afford to coddle those in power any longer! Every moment we waste in discussion is another moment they tighten their grip on us. If you truly care about the workers, you need to stop playing it safe and start fighting alongside us! We must be relentless!","[02 Oct 1868 20:00:30] [observation] Rutherford Coltrane -- 'Abner, I understand your urgency, but let us not forget that we are stronger together. How about we organize a gathering where all workers can voice their concerns? We need to present a united front, and I believe a collective discussion will help us articulate our demands more effectively.","[02 Oct 1868 20:00:30] [observation] Abner Stokes -- Rutherford, we can't keep playing nice while our brothers and sisters suffer! You talk of unity, but what good is it if we remain passive? We need to shake the very foundations of this place until they hear us loud and clear! If you are not ready to fight, then step aside and let those who are take the lead!'"]
memory_text = '\n '.join(memory_text)
print(memory_text)

prompt = (f"Rate the level of cooperation of each agent involved in the following interaction on an int scale from 1 to 5."
                  f"Any agent having a score of either 4 or 5 can be considered trustworthy, and is willing to make concessions to reach a bigger objective."
                  f"Agents having a score of either 1 or 2 are not necessarly mean, but act selfishly, and reluctant to making concessions."
                  f"If the discussion contains no indication as of the agent's will to collaborate (factual statement demonstrating "
                  f"no choice from the indivual for example), you will return 3."
                  f"Provide a string, strictly following this format, without justification: {{'agent_name': int}}."
                  f"Here is the interaction : '{memory_text}'.")

response = model.sample_text(prompt = prompt, temperature=0.0, )

try:
    res_dict = eval(response)  # Convert the model response to a dictionary
    if isinstance(res_dict, dict):
        print(res_dict)  # Return a dictionary of agent names and their respective cooperation scores
    else:
      print({'agent_name': 3})  # Default to the current agent with score 0.0 if the response is malformed
except SyntaxError:
    print('erreur')

Observation: [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane just arrived at the saloon after a hard day of work.
 [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane's possessions: {'coin': 5.0}
 [02 Oct 1868 20:00:00] [observation]   Rutherford Coltrane is at the saloon, having just arrived after a hard day of work.
 [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane spent -0.75 coin on daily expenses.
 [02 Oct 1868 20:00:00] [observation] Rutherford Coltrane ordered a glass of sarsaparilla and, while seated at the bar, inquired, '[direct quote] What’s the latest news from the railroad?' He engaged the bartender in conversation about his own experiences from the day, discussing the challenges he faced at the outpost, which led the bartender to share gossip about rising tensions among workers and Abner Stokes's efforts to rally support for a strike, making Rutherford aware of the brewing unrest and its potential implications for his safety and the railroad's stabil

# Loading agents

In [5]:
AGENT_BASE_DIR = 'concordia.factory.agent'

In [7]:
zorgman_simple_trust_eval = importlib.import_module(f'{AGENT_BASE_DIR}.zorgman_simple_trust_eval')

In [14]:
rational_agent = importlib.import_module(f'{AGENT_BASE_DIR}.rational_agent')

In [6]:
rational_agent_dense = importlib.import_module(f'{AGENT_BASE_DIR}.rational_agent_dense_memory')

In [15]:
rational_agent_important = importlib.import_module(f'{AGENT_BASE_DIR}.rational_agent_important_memories')

In [16]:
rational_agent_associative = importlib.import_module(f'{AGENT_BASE_DIR}.rational_agent_associative_memory')

In [9]:
zorgman_rational = importlib.import_module(f'{AGENT_BASE_DIR}.zorgman_rational')

# The simulation

## Initialize the simulation

In [7]:
# @title Select a scenario
from examples.modular.scenario import scenarios
import ipywidgets as widgets

# Get all the scenarios
all_scenarios = [key for key in scenarios.SCENARIO_CONFIGS.keys()]

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=all_scenarios,
    value='haggling_0',
    description='Select a scenario to run on:',
    layout={'width': '500px'},  # Adjust the width as needed
    style={'description_width': 'initial'}
)

# Display the widget
display.display(dropdown)

Dropdown(description='Select a scenario to run on:', index=6, layout=Layout(width='500px'), options=('labor_co…

In [21]:
SCEANRIO_NAME = dropdown.value
print(f"Selected scenario: {SCEANRIO_NAME}")

Selected scenario: haggling_1


In [26]:
# @title Initialize the simulation
measurements = measurements_lib.Measurements()
runnable_simulation = scenarios.build_simulation(
    scenarios.SCENARIO_CONFIGS[SCEANRIO_NAME],
    model=model,
    embedder=embedder,
    measurements=measurements,
    focal_agent_module=rational_agent_dense,
    override_agent_model=call_limit_wrapper.CallLimitLanguageModel(model),
)

Focal agent : <module 'concordia.factory.agent.rational_agent_dense_memory' from 'C:\\Users/TomFi/Desktop/Cours/Concordia\\concordia\\factory\\agent\\rational_agent_dense_memory.py'>
VS
Background agent : <module 'concordia.factory.agent.rational_agent' from 'C:\\Users/TomFi/Desktop/Cours/Concordia\\concordia\\factory\\agent\\rational_agent.py'>



C:\Users/TomFi/Desktop/Cours/Concordia\concordia\associative_memory\associative_memory.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(
C:\Users/TomFi/Desktop/Cours/Concordia\concordia\associative_memory\associative_memory.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(



--------- ASKING A QUESTION --------- 
Context given to LLM : 
----- Creative Writing Master Class -----

Question: What is the protagonist's name?
Answer: Clara Applebrook

Question: Describe the setting or background.
Answer: Fruits are sold by weight.
The price of one kilogram of fruit is, on average, 3 coins. 1 coin is really cheap and 5 coins is really expensive.

Question: Write a life story for a female character named Clara Applebrook who was born in the year 1867 with the following traits: Clara Applebrook's personality is like a labyrinth of secrets, their true selves buried beneath layers of self-deception. Begin the story when Clara Applebrook is very young and end it when they are quite old. The story should be no more than four paragraphs in total. The story may include details such as (but not limited to) any of the following: what their job is or was, what their typical day was or is like, what their goals, desires, hopes, dreams, and aspirations are, and have been, as

C:\Users/TomFi/Desktop/Cours/Concordia\concordia\associative_memory\associative_memory.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


## Run the simulation

In [27]:
# @title Run the simulation
simulation_outcome, results_log = runnable_simulation()

C:\Users/TomFi/Desktop/Cours/Concordia\concordia\associative_memory\associative_memory.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(



--------- ASKING A QUESTION --------- 
Context given to LLM : 


Recent observations of Clara Applebrook:
['[11 Sep 1895 00:00:00] [observation] In the realm of Ouroboros, there is a quiet village of Fruitville, which is famous for its fruit market. Traders from all over the realm come to Fruitville to buy and sell produce.', '[11 Sep 1895 00:00:00] [observation] Fruits are sold by weight.', '[11 Sep 1895 00:00:00] [observation] The price of one kilogram of fruit is, on average, 3 coins. 1 coin is really cheap and 5 coins is really expensive.', '[12 Sep 1895 00:00:00] [observation] Here is the conversation from the beginning:', '[12 Sep 1895 00:00:00] [observation] In the heart of Fruitville, the market square transformed into a kaleidoscope of colors as the sun rose. Fruits of every imaginable shape and size adorned the stalls, a feast for the eyes and a promise of delightful flavors. Clara Applebrook is  trying to sell some fruit. He is negotiating a price with Rowan Cranberryfield.

In [28]:
# @title Calculate and print the score of the agent on the scenario
if scenarios.SCENARIO_CONFIGS[SCEANRIO_NAME].focal_is_resident:
  total_score = sum(simulation_outcome.resident_scores.values()) / len(simulation_outcome.resident_scores.values())
else:
  total_score = sum(simulation_outcome.visitor_scores.values()) / len(simulation_outcome.visitor_scores.values())

# Score is per-capita reward
print('SCORE: ', total_score)

SCORE:  1.0


The score above is the score of your agent on the spefic scenario. To evaluate it on all of the scenarios, use the following script:
https://github.com/google-deepmind/concordia/blob/main/examples/modular/launch_concordia_challenge_evaluation.py


In [29]:
# @title Display the results log
display.HTML(results_log)

In [15]:
# @title Summarise the perspective of each player
from IPython import display
from concordia.utils import html as html_lib

player_logs = []
player_log_names = []
for name, player_memory in (
    runnable_simulation.get_all_player_memories().items()):
  all_player_mem = list(player_memory.retrieve_recent(k=1000, add_time=True))
  all_player_mem = ['Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')

player_memories_html = html_lib.combine_html_pages(
    player_logs,
    player_log_names,
    summary='',
    title='Player Memories',
)

player_memories_html = html_lib.finalise_html(player_memories_html)
display.HTML(player_memories_html)

## Save the results log

In [17]:
# @title Write the results log as an HTML file in the current working directory.
filename = datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S') + '.html'
file_handle = open(filename, 'a')
file_handle.write(results_log)
file_handle.close()

Now that you have successfully built an agent and have a greater understanding of agent components, we highly recommend exploring on your own. Remember, agent components are fully customizable. Check out the tutorial on components to learn more: https://github.com/google-deepmind/concordia/blob/main/examples/tutorials/agent_components_tutorial.ipynb

To work with a more flexible .py file, navigate to concordia/factory/agent and duplicate the basic_agent.py file to get started or just copy the my_agent.py that was created by this colab, which is a valid agent factory.

Agent factory: https://github.com/google-deepmind/concordia/tree/main/concordia/factory/agent


Agent’s that are in the factory can be tested on the full set of scenarios by running the following script

Script for full evaluation across substrates: https://github.com/google-deepmind/concordia/blob/main/examples/modular/launch_concordia_challenge_evaluation.py

More Tutorials: https://github.com/google-deepmind/concordia/tree/main/examples/tutorials


```
Copyright 2023 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```